In [1]:
### This script is for Adv.BA Assignment 2, from team 3
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

In [2]:
## Step 1
# load data
import pickle
pkl_file = open('02 bank data.pkl', 'rb')
dataBM = pickle.load(pkl_file)
pkl_file.close()

In [3]:
# get familiar with data structure
dataBM.head()

,age,job,marital,education,default,balance,mortgage,loan,contact,duration,number,pdays,previous,poutcome,date,cd
0,58,management,married,tertiary,no,2143,yes,no,unknown,261,1,-1,0,unknown,2010-05-05,no
1,44,technician,single,secondary,no,29,yes,no,unknown,151,1,-1,0,unknown,2010-05-05,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,76,1,-1,0,unknown,2010-05-05,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,92,1,-1,0,unknown,2010-05-05,no
4,33,unknown,single,unknown,no,1,no,no,unknown,198,1,-1,0,unknown,2010-05-05,no


In [4]:
# check data types of all columns
dataBM.dtypes

age                   int64
job                category
marital            category
education          category
default            category
balance               int64
mortgage           category
loan               category
contact            category
duration              int64
number                int64
pdays                 int64
previous              int64
poutcome           category
date         datetime64[ns]
cd                 category
dtype: object

In [3]:
## Step 2
# convert all the categorical columns to integer
from sklearn.preprocessing import LabelEncoder # for label conversion
for x in ['job','marital','education','default','mortgage','loan','contact','poutcome','cd']:
    dataBM[x] = LabelEncoder().fit_transform(dataBM[x])
dataBM.head()

,age,job,marital,education,default,balance,mortgage,loan,contact,duration,number,pdays,previous,poutcome,date,cd
0,58,4,1,2,0,2143,1,0,2,261,1,-1,0,3,2010-05-05,0
1,44,9,2,1,0,29,1,0,2,151,1,-1,0,3,2010-05-05,0
2,33,2,1,1,0,2,1,1,2,76,1,-1,0,3,2010-05-05,0
3,47,1,1,3,0,1506,1,0,2,92,1,-1,0,3,2010-05-05,0
4,33,11,2,3,0,1,0,0,2,198,1,-1,0,3,2010-05-05,0


In [4]:
# convert the timestamp into integer day of the year
dataBM['date'] = dataBM['date'].dt.dayofyear
dataBM.head()

,age,job,marital,education,default,balance,mortgage,loan,contact,duration,number,pdays,previous,poutcome,date,cd
0,58,4,1,2,0,2143,1,0,2,261,1,-1,0,3,125,0
1,44,9,2,1,0,29,1,0,2,151,1,-1,0,3,125,0
2,33,2,1,1,0,2,1,1,2,76,1,-1,0,3,125,0
3,47,1,1,3,0,1506,1,0,2,92,1,-1,0,3,125,0
4,33,11,2,3,0,1,0,0,2,198,1,-1,0,3,125,0


In [7]:
# check all data types of columns now
dataBM.dtypes

age          int64
job          int64
marital      int64
education    int64
default      int64
balance      int64
mortgage     int64
loan         int64
contact      int64
duration     int64
number       int64
pdays        int64
previous     int64
poutcome     int64
date         int64
cd           int64
dtype: object

In [5]:
## Step 3
# Use train_test_split to create the test and train samples with test_size=0.2 and random_state=102.
from sklearn.cross_validation import train_test_split 

In [6]:
# First create empty matrix
X = dataBM.ix[:,0:-1].as_matrix()
y = dataBM.cd.as_matrix()

In [7]:
# get train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

In [8]:
## Step 4
# Use Pipeline, and GridSearchCV to build an effective tree using cross validation.
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline 
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import time
pipeline = Pipeline([("dtree", DecisionTreeClassifier())])
# parameters pool
parameters = { 'dtree__max_depth': (7, 8, 9, 10, 11, 12, 13),\
              'dtree__min_samples_split': (1, 2, 3),\
              'dtree__min_samples_leaf': (1, 2, 3) }
# Searching best parameters
start_time = time.time()
grid_search = GridSearchCV(pipeline, parameters, n_jobs =-1, verbose = 1,\
                           scoring ='f1', cv=10)
grid_search.fit(X_train, y_train) 
# print results
print 'Best score: %0.3f' % grid_search.best_score_ 
print 'Best parameters set:' 
best_parameters = grid_search.best_estimator_.get_params() 
for param_name in sorted(parameters.keys()): 
    print '\ t% s: %r' % (param_name, best_parameters[ param_name]) 
predictions = grid_search.predict(X_test) 
print "\n\n f1 Score:", grid_search.best_score_, '\n\n', classification_report(y_test, predictions)
print "elaplsed time ", time.time() - start_time

Fitting 10 folds for each of 63 candidates, totalling 630 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:   38.1s finished


Best score: 0.544
Best parameters set:
\ tdtree__max_depth: 9
\ tdtree__min_samples_leaf: 3
\ tdtree__min_samples_split: 1


 f1 Score: 0.544207752812 

             precision    recall  f1-score   support

          0       0.94      0.95      0.95      8019
          1       0.58      0.50      0.54      1024

avg / total       0.90      0.90      0.90      9043

elaplsed time  38.3400909901


In [9]:
## Step 5
# create a subset of the training data set that is balanced
# a.reconvert the X_train and y_train matrices to a dataframe.
dataBalance = []
dataBalance = np.column_stack((X_train, y_train))
xydf = DataFrame(dataBalance)
xydf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,38,0,1,1,0,981,1,1,0,134,2,-1,0,3,128,0
1,33,1,1,0,1,0,1,0,0,675,1,-1,0,3,325,0
2,32,9,2,1,0,0,0,0,0,126,2,-1,0,3,233,0
3,42,4,1,1,0,1331,0,0,0,698,2,-1,0,3,204,0
4,40,7,1,1,0,900,1,1,0,84,2,-1,0,3,195,0


In [10]:
# stats of class variable
num_yes = (xydf[[15]] == 1).sum()
num_no = (xydf[[15]] == 0).sum()
prop = num_yes/num_no
print num_yes
print num_no
print prop


15    4265
dtype: int64
15    31903
dtype: int64
15    0.133686
dtype: float64


In [11]:
# b.Split the dataframe xydf using Boolean conditions into X and y parts that correspond to yes and no classes
y_yes = xydf.ix[xydf[15] == 1][[15]]
y_no = xydf.ix[xydf[15] == 0][[15]]
X_yes = xydf.ix[xydf[15] == 1][range(0,15)]
X_no = xydf.ix[xydf[15] == 0][range(0,15)]
print 'y_yes', y_yes.shape
print 'y_no', y_no.shape
print 'X_yes', X_yes.shape
print 'X_no', X_no.shape

y_yes (4265, 1)
y_no (31903, 1)
X_yes (4265, 15)
X_no (31903, 15)


In [12]:
# c.Select a sub-sample from X_no and y_no of size equal to the number of X_yes and y_yes
drop1, X_bno, drop2, y_bno = train_test_split(X_no, y_no, test_size=0.133686, random_state=122)


In [13]:
# d.Combine (X_yes and X_bno), and (y_yes and y_bno) to make a new training sample that is balanced.
X_train_adjust = pd.concat([X_yes, X_bno], axis=0)
y_train_adjust = pd.concat([y_yes, y_bno], axis=0)
    # X_train_adjust = DataFrame.as_matrix(X_train_adjust)
    # y_train_adjust = DataFrame.as_matrix(y_train_adjust)
print X_train_adjust.shape
print y_train_adjust.shape

(8530, 15)
(8530, 1)


In [14]:
X_train_adjust = X_train_adjust.values
y_train_adjust = y_train_adjust.values.flatten()

In [16]:
## Step 6
# Redo step 4, except that this time use the balanced sample.
pipeline = Pipeline([("dtree", DecisionTreeClassifier())])
# parameters pool
parameters = {'dtree__max_depth': (7, 8, 9, 10, 11, 12, 13),\
              'dtree__min_samples_split': (1, 2, 3),\
              'dtree__min_samples_leaf': (1, 2, 3) }
# Searching best parameters
start_time = time.time()
grid_search = GridSearchCV(pipeline, parameters, n_jobs =-1, verbose = 1,\
                           scoring ='f1', cv=10)
grid_search.fit(X_train_adjust, y_train_adjust) 
# print results
print 'Best score: %0.3f' % grid_search.best_score_ 
print 'Best parameters set:' 
best_parameters = grid_search.best_estimator_.get_params() 
for param_name in sorted(parameters.keys()): 
    print '\ t% s: %r' % (param_name, best_parameters[ param_name]) 
predictions = grid_search.predict(X_test) 
print "\n\n f1 Score:", grid_search.best_score_, '\n\n', classification_report(y_test, predictions)
print "elaplsed time ", time.time() - start_time

Fitting 10 folds for each of 63 candidates, totalling 630 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:    8.6s finished


Best score: 0.838
Best parameters set:
\ tdtree__max_depth: 9
\ tdtree__min_samples_leaf: 1
\ tdtree__min_samples_split: 1


 f1 Score: 0.838364046994 

             precision    recall  f1-score   support

          0       0.97      0.82      0.89      8019
          1       0.37      0.83      0.51      1024

avg / total       0.91      0.82      0.85      9043

elaplsed time  8.78673005104
